In [1]:
# Imports
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pickle
from libHanSolo import unpacker
from matplotlib import pyplot

In [2]:
# Constants
INDEX_ORDER = [(x, x + 1024, x + 2048) for x in range(0, 1024)]
INPUT_SHAPE = (32, 32, 3)

In [3]:
def DataToImage(pixel_values):
    ordered = pixel_values[INDEX_ORDER]
    return np.array(ordered.reshape(INPUT_SHAPE))

def BatchToImages(batch):
    images = unpacker.get_value(batch, 'data')
    return np.array([DataToImage(image) for image in images])

def LabelToVector(label_value):
    arr = np.zeros(10)
    arr[label_value] = 1.0
    return arr

def BatchToLabelVector(batch):
    labels = unpacker.get_value(batch, 'labels')
    return np.array([LabelToVector(label) for label in labels])

def SetupData():
    batches = [unpacker.unpickle(f'./data/data_batch_{index}') for index in range(1, 6)]
    test = unpacker.unpickle(f'./data/test_batch')
    X_train = np.concatenate([BatchToImages(batch) for batch in batches])
    X_test = BatchToImages(test)
    y_train = np.concatenate([BatchToLabelVector(batch) for batch in batches])
    y_test = BatchToLabelVector(test)
    return [X_train, y_train, X_test, y_test]

In [4]:
[X_train, y_train, X_test, y_test] = SetupData()
print(f'{X_train.shape} {X_test.shape} {y_train.shape} {y_test.shape}')

(50000, 32, 32, 3) (10000, 32, 32, 3) (50000, 10) (10000, 10)


In [93]:
base_model = keras.applications.resnet50.ResNet50(weights="imagenet", include_top=False, input_shape=INPUT_SHAPE, pooling='avg')
output_layer = keras.models.Sequential([
    keras.layers.Dense(512, activation="relu"),
    keras.layers.Dense(10, activation="softmax")
])(base_model.output)

model = keras.models.Model(inputs=base_model.input, outputs=output_layer)
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

def process(set):
    return keras.applications.resnet.preprocess_input(set)
    
X_train_processed = process(X_train)
X_test_processed = process(X_test)

In [104]:
model.fit(X_train_processed, y_train)
loss, score = model.evaluate(X_test_processed, y_test)

Model: "model_15"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 38, 38, 3)    0           input_5[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 16, 16, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 16, 16, 64)   256         conv1_conv[0][0]                 
___________________________________________________________________________________________